In [1]:
import numpy as np
import pandas as pd

import bebi103
import cmdstanpy
import arviz as az

import bokeh.plotting
import bokeh.io
from bokeh.io import output_file, save
bokeh.io.output_notebook()

/home/max/micromamba/envs/pol_stats/lib/python3.12/site-packages/bebi103/viz.py:38: UserWarning: DataShader import failed with error "cannot import name 'Callable' from 'collections' (/home/max/micromamba/envs/pol_stats/lib/python3.12/collections/__init__.py)".
Features requiring DataShader will not work and you will get exceptions.
  warnings.warn(


Loading BokehJS ...

$\dot{A}=gA$ 

$\dot{g}=-\alpha(g-\beta \frac{A_{end}-A}{A_{end}})$ if $A<A_{cut}$

Combine Dev and Reg with differnt initial conditions

In [2]:
rng = np.random.default_rng()

from scipy.integrate import solve_ivp

def A_theor(t, A_0, g_0, alpha, beta, A_end,A_cut):
    t48 = t - 48

    def ode_system(t, y):
        A, g = y
        dAdt = g*A
        if A < A_cut:
            dgdt = -alpha * (g - beta * (A_end - A_cut) / A_end)
        else:
            dgdt = -alpha * (g - beta * (A_end - A) / A_end)
        return [dAdt, dgdt]

    # Initial conditions
    y0 = [A_0, g_0]

    # Solve the ODE
    solution = solve_ivp(ode_system, [t48.min(), t48.max()], y0, t_eval=t48, method='RK45')
    #print("Solver message:", solution.message)
    return solution.y[0]

def linCrit_ppc(x):
    sigma = 0*np.abs(rng.normal(0, 2.0))
    A_0 = 10**rng.normal(0.3, 0.15)
    g_0 = np.abs(rng.normal(0, 0.1))
    alpha = 10**rng.normal(-0.25, 0.25)
    beta = (alpha/4)*10**rng.normal(0, 0.1)
    A_end = 10**rng.normal(1.0, 0.1)
    A_cut = 2 + 4*rng.beta(5.0,5.0)
    #print(A_0, g_0,alpha, beta, A_end)
    return (rng.normal(A_theor(x, A_0, g_0,alpha, beta, A_end,A_cut), sigma)/A_end,
            {"A_0": A_0, "g_0": g_0, "alpha": alpha, "beta": beta, "A_end": A_end, "A_cut":A_cut})

In [3]:
x = np.linspace(48, 144, 30)
for _ in range(10):
    print(linCrit_ppc(x))

(array([0.37757356, 0.76467726, 0.97390719, 0.9993443 , 1.00007351,
       1.00000707, 1.00000003, 1.00000017, 0.99999977, 0.9999993 ,
       1.00000018, 0.99999983, 0.99999943, 0.99999963, 1.00000147,
       1.        , 0.99999996, 1.00000031, 0.99999943, 0.99999999,
       1.00000034, 1.00000134, 0.99999824, 1.00000019, 0.99999991,
       1.00000016, 1.00000027, 0.99999915, 1.00000156, 1.00000014]), {'A_0': 2.796646467904989, 'g_0': np.float64(0.1641990682667403), 'alpha': 1.6439785234466162, 'beta': 0.4790439778730888, 'A_end': 7.406891612153401, 'A_cut': 3.0321394350563167})
(array([0.1387052 , 0.15352867, 0.18220311, 0.22182678, 0.27262096,
       0.33623315, 0.41476903, 0.50531175, 0.60131309, 0.6953679 ,
       0.78050834, 0.85148162, 0.90619705, 0.94489479, 0.97035506,
       0.98576081, 0.99437084, 0.99868172, 1.00054066, 1.00111258,
       1.00110146, 1.00087461, 1.00060357, 1.00038756, 1.00022533,
       1.00012112, 1.00005873, 1.00002235, 1.00000534, 0.99999796]), {'A_0': 1

In [4]:
x = np.linspace(48, 144, 100)
f_ppc = [linCrit_ppc(x) for _ in range(100)]

p = bokeh.plotting.figure(
    x_axis_label="x",
    y_axis_label="f",
    frame_height=400,
    frame_width=400,
)

for f, params in f_ppc:
    line = p.line(x, f)
    p.add_tools(bokeh.models.HoverTool(
        renderers=[line],
        tooltips=[(key, f"{params[key]}") for key in params.keys()]
    ))

bokeh.io.show(p)

In [5]:
import os
Sahra_Shivani_path=(r'/home/max/Documents/02_Data/sahra_shivani_data')
Output_path=os.path.join(Sahra_Shivani_path,'sorted_data')
Curv_Thick_path=os.path.join(Output_path,'for_curv_thick')
hdf5_file_path = os.path.join(Curv_Thick_path, 'scalarGrowthData.h5')

# Aoad the DataFrame from the HDF5 file
df = pd.read_hdf(hdf5_file_path, key='data')

p = bokeh.plotting.figure(
    frame_width=350,
    frame_height=250,
    x_axis_label="time in hpf",
    y_axis_label="Surface Area",
    #x_axis_type="log",
    toolbar_location='above',
)

colors = bokeh.palettes.Category10_3
items = []
for color, (genotype, sub_df) in zip(colors, df.groupby("condition")):
    items.append(
        (
            bokeh.models.LegendItem(
                label=genotype,
                renderers=[
                    p.scatter(
                        sub_df["time in hpf"],
                        sub_df["Surface Area"]*1e-4,
                        color=color,
                    )
                ],
            )
        )
    )

p.add_layout(bokeh.models.Legend(items=items), 'right')
bokeh.io.show(p)

In [6]:
sm = cmdstanpy.CmdStanModel(stan_file='fit_area.stan')

In [7]:
def df_to_data(cat_Dev,cat_Reg):
    filtered_df = df[df['condition'] == cat_Reg]
    t_Reg = filtered_df['time in hpf'].to_numpy()
    A_Reg = filtered_df['Surface Area'].to_numpy()*1e-4
    sorted_indices = np.argsort(t_Reg)
    t_Reg = t_Reg[sorted_indices]
    A_Reg = A_Reg[sorted_indices]

    filtered_df = df[df['condition'] == cat_Dev]
    t_Dev = filtered_df['time in hpf'].to_numpy()
    A_Dev = filtered_df['Surface Area'].to_numpy()*1e-4
    sorted_indices = np.argsort(t_Dev)
    t_Dev = t_Dev[sorted_indices]
    A_Dev = A_Dev[sorted_indices]

    N_ppc = 100
    t_ppc = np.linspace(48, 144, N_ppc)
    
    return dict(t_Reg=t_Reg, A_Reg=A_Reg, N_Reg=len(t_Reg),
                t_Dev=t_Dev, A_Dev=A_Dev, N_Dev=len(t_Dev),
                  N_ppc=N_ppc, t_ppc=t_ppc)

data = df_to_data('Development','Regeneration')

In [8]:
samples = sm.sample(data=data,iter_warmup=3000,iter_sampling=1000,show_console = False )
samples = az.from_cmdstanpy(posterior=samples, posterior_predictive=['A_Dev_ppc','A_Reg_ppc'])
bebi103.stan.check_all_diagnostics(samples)

18:20:00 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

18:21:16 - cmdstanpy - INFO - CmdStan done processing.
18:21:16 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: Exception: ode_rk45: ode parameters and data[1] is inf, but must be finite! (in 'fit_area.stan', line 28, column 8 to column 91) (in 'fit_area.stan', line 107, column 4 to column 96)
	Exception: Exception: ode_rk45: ode parameters and data[1] is inf, but must be finite! (in 'fit_area.stan', line 28, column 8 to column 91) (in 'fit_area.stan', line 107, column 4 to column 96)
	Exception: Exception: ode_rk45: ode parameters and data[2] is inf, but must be finite! (in 'fit_area.stan', line 28, column 8 to column 91) (in 'fit_area.stan', line 107, column 4 to column 96)
	Exception: normal_lpdf: Location parameter[1] is -nan, but must be finite! (in 'fit_area.stan', line 107, column 4 to column 96)
	Exception: Exception: ode_rk45:  Failed to integrate to next output time (48) in less than max_num_steps steps (in 'fit_area.stan', line 28, column 8 to column 91) 


Effective sample size looks reasonable for all parameters.

Rhat looks reasonable for all parameters.

0 of 4000 (0.0%) iterations ended with a divergence.

0 of 4000 (0.0%) iterations saturated the maximum tree depth of 10.

E-BFMI indicated no pathological behavior.


0

In [12]:
output_file("area_DevREG_ppc_corner_Dev.html")
bokeh.io.show(
    bebi103.viz.corner(
        samples,
        parameters=['A_end_Dev','A_cut_Dev','A_0_Dev','g_0_Dev', 'alpha','beta_'],
        xtick_label_orientation=np.pi/4,
        plot_ecdf=False
    )
)

In [13]:
output_file("area_DevREG_ppc_corner_Reg.html")
bokeh.io.show(
    bebi103.viz.corner(
        samples,
        parameters=['A_end_Reg','A_cut_Reg','A_0_Reg','g_0_Reg', 'alpha','beta_'],
        xtick_label_orientation=np.pi/4,
        plot_ecdf=False
    )
)

In [ ]:
samples

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats

In [14]:
output_file("area_DevREG_ppc_At_Dev.html")
f_ppc = samples.posterior_predictive.A_Dev_ppc.stack(
    {"sample": ("chain", "draw")}
).transpose("sample", "A_Dev_ppc_dim_0")

bokeh.io.show(
    bebi103.viz.predictive_regression(
        f_ppc,
        samples_x=np.array(data["t_ppc"]),
        data=np.dstack((data["t_Dev"], data["A_Dev"])).squeeze(),
        #x_axis_type="log",
        x_axis_label='t',
        y_axis_label='A',
    )
)

In [15]:
output_file("area_DevREG_ppc_At_Reg.html")
f_ppc = samples.posterior_predictive.A_Reg_ppc.stack(
    {"sample": ("chain", "draw")}
).transpose("sample", "A_Reg_ppc_dim_0")

bokeh.io.show(
    bebi103.viz.predictive_regression(
        f_ppc,
        samples_x=np.array(data["t_ppc"]),
        data=np.dstack((data["t_Reg"], data["A_Reg"])).squeeze(),
        #x_axis_type="log",
        x_axis_label='t',
        y_axis_label='A',
    )
)

In [17]:
posterior = samples.posterior

# Dictionary to store mean and confidence intervals for each variable
results ={}



# Loop through each variable in the posterior
for var_name in posterior.data_vars:
    data = posterior[var_name].values.flatten()
    
    results[var_name]=data

print(results)
# Convert the results dictionary to a pandas DataFrame
results_df = pd.DataFrame(results)

# Display the table
print(results_df)
Output_path=os.path.join(Sahra_Shivani_path,'sorted_data')
Curv_Thick_path=os.path.join(Output_path,'for_curv_thick')
results_df.to_csv(os.path.join(Curv_Thick_path,"area_sampled_parameter_results.csv"), index=False)

{'A_0_Dev': array([2.45682, 1.63604, 1.90564, ..., 2.31831, 1.97607, 2.03045]), 'A_0_Dev_tilde': array([0.390374, 0.213794, 0.28004 , ..., 0.365172, 0.295803, 0.307593]), 'A_0_Reg': array([1.27999, 1.66903, 1.46633, ..., 1.50488, 1.47467, 1.76069]), 'A_0_Reg_tilde': array([0.107207, 0.222465, 0.166231, ..., 0.177503, 0.168695, 0.245684]), 'A_cut_Dev': array([4.71703, 3.26547, 4.36364, ..., 3.75413, 4.57782, 3.64147]), 'A_cut_Dev_tilde': array([ 0.750356, -0.770511,  0.36773 , ..., -0.247116,  0.594756,
       -0.362444]), 'A_cut_Reg': array([4.76717, 3.99637, 4.1081 , ..., 2.77185, 3.8623 , 4.30435]), 'A_cut_Reg_tilde': array([ 0.808516  , -0.00363358,  0.108205  , ..., -1.43088   ,
       -0.137918  ,  0.306733  ]), 'A_end_Dev': array([10.6289, 11.4607, 10.7308, ..., 10.883 , 11.112 , 10.939 ]), 'A_end_Dev_tilde': array([1.02649, 1.05921, 1.03063, ..., 1.03675, 1.04579, 1.03898]), 'A_end_Reg': array([12.3724, 12.9082, 11.3021, ..., 11.8946, 12.197 , 11.9377]), 'A_end_Reg_tilde': array

In [ ]:
bebi103.stan.clean_cmdstan()